In [1]:

import re
import pandas as pd

def parse_cam_log(fname):
    logheaderkey = '# Log header:'
    comments = []
    with open(fname,'r') as fd:
        for line in fd:
            if line.startswith('#'):
                line = line.strip('\n').strip('\r')
                comments.append(line)
                if line.startswith(logheaderkey):
                    columns = line.strip(logheaderkey).strip(' ').split(',')

    logdata = pd.read_csv(fname,names = columns, 
                          delimiter=',',
                          header=None,
                          comment='#',
                          engine='c')
    return logdata,comments

def parse_log_comments(comm,msg = 'trial_start:',strkeyname = 'i_trial',strformat=int):
    # this will get the start frames for each trial
    comm = map(lambda x: x.strip('#').replace(' ',''),comm)
    commmsg = list(filter(lambda x:msg in x ,comm))
    table = []
    for i,lg in enumerate(commmsg):
        lg = re.split(',|-|'+msg,lg)
        table.append({strkeyname:strformat(lg[-1]),
                         'iframe':int(lg[0]),
                         'timestamp':float(lg[1])})
    return pd.DataFrame(table)



In [4]:
fname = '/home/couto/data/JC002/SpatialSparrow/Session Data/JC002_SpatialSparrow_20200318_094828_facecam_00000000.camlog'

log,comm = parse_cam_log(fname)
start_trials = parse_log_comments(comm)
end_trials = parse_log_comments(comm,'trial_end:')
# merge start and end 
start_trials = start_trials.rename(columns={'iframe':'start_frame','timestamp':'start_timestamp'})
end_trials = end_trials.rename(columns={'iframe':'end_frame','timestamp':'end_timestamp'})
trialinfo = pd.merge(start_trials,end_trials)
trialinfo

,i_trial,start_frame,start_timestamp,end_frame,end_timestamp
0,1,282,582606.553632,458,582610.081793
1,2,574,582612.407169,753,582615.995465
2,3,875,582618.441120,1066,582622.269973
3,4,1190,582624.755718,1340,582627.762673
4,5,1473,582630.428837,1642,582633.816670
...,...,...,...,...,...
405,406,140936,585426.152414,141179,585431.023676
406,407,141355,585434.551833,141493,585437.318229
407,408,141798,585443.432364,142075,585448.985202
408,409,142273,585452.954378,142542,585458.346845
